Ваша задача - построить алгоритм, определяющий вероятность дефолта того или иного заказа на исторических данных из таверн различных гномьих общин с 2015-02-24 по 2016-09-30.

# Загрузка

In [24]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
import time
import datetime

In [4]:
train = pd.read_csv('train.csv')

In [5]:
test = pd.read_csv('test.csv')

In [6]:
train

,Deal_id,Deal_date,First_deal_date,Secret_dwarf_info_1,Secret_dwarf_info_2,Secret_dwarf_info_3,First_default_date,Successful_deals_count,Region,Tavern,Hashed_deal_detail_1,Hashed_deal_detail_2,Hashed_deal_detail_3,Hashed_deal_detail_4,Hashed_deal_detail_5,Hashed_deal_detail_6,Age,Gender,Default
0,22487461,2015-11-05,2015-08-29,NaN,NaN,NaN,NaN,0.0,Tavern_district_3,7,2.5,-3,8,2.5,-3,5,36.0,Male,0
1,62494261,2016-08-26,2015-12-21,3.5,-2.0,5.0,2016-07-30,2.0,Tavern_district_4,7,2.5,-3,14,3.5,-3,5,29.0,Female,1
2,34822849,2016-02-18,2015-11-11,NaN,NaN,NaN,NaN,0.0,Tavern_district_6,7,2.5,-3,8,2.5,-3,5,56.0,Female,0
3,46893387,2016-04-30,2016-03-22,NaN,NaN,NaN,NaN,0.0,Tavern_district_2,13,2.5,-2,5,2.5,-3,5,27.0,Female,0
4,67128275,2016-09-19,2016-07-21,NaN,NaN,NaN,NaN,0.0,Tavern_district_4,39,2.5,-3,7,2.5,-3,5,37.0,Female,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,42976693,2016-04-09,2015-11-07,NaN,NaN,NaN,NaN,0.0,Tavern_district_3,7,2.5,-3,10,2.5,-3,5,37.0,Female,0
3164,50503195,2016-05-23,2015-12-17,NaN,NaN,NaN,NaN,0.0,Tavern_district_5,7,2.5,-3,10,2.5,-3,5,31.0,Female,0
3165,66034984,2016-09-14,2016-08-15,NaN,NaN,NaN,NaN,0.0,Tavern_district_4,7,2.5,-3,5,3.5,-3,5,26.0,Male,1
3166,65042945,2016-09-09,2016-03-14,NaN,NaN,NaN,NaN,0.0,Tavern_district_7,7,2.5,-3,11,2.5,-3,5,28.0,Female,0


In [8]:
catboost = CatBoostClassifier(iterations=80
                            #   , learning_rate=1
                            #   , depth=2
                              )


In [9]:
X_train = train.drop(['Deal_id', 'Default'], axis=1)
y_train = train['Default']

In [23]:
train['had_default'] = np.where(train[train.First_default_date.isnull()], 1, 0)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [25]:
time_columns = ['Deal_date', 'First_deal_date', 'First_default_date']

def convert_to_timestamp(s):
        time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d").timetuple())

for col in time_columns:
    X_train[col] = X_train[col].apply(convert_to_timestamp)


TypeError: strptime() argument 1 must be str, not None

In [10]:



catboost.fit(X_train,
             y_train,
             cat_features=[
                           'Region',
                           'Tavern',
                           'Gender'],
            #  text_features=['text'],
             verbose=False,
             plot=True
             )

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="2015-11-05": Cannot convert 'b'2015-11-05'' to float